In [1]:
import openai
import gradio as gr
import os
from gtts import gTTS
import subprocess
import tempfile
import dotenv
dotenv.load_dotenv()
from pygame import mixer

openai.api_key = os.getenv('OPENAI_API_KEY')

messages=[
  {"role":"system","content":"You are a helpful therapist"},
]


pygame 2.2.0 (SDL 2.0.22, Python 3.11.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:

def say(text,filename=None):
  with tempfile.NamedTemporaryFile(delete=True) as temp:
    tts = gTTS(text=text,lang='en',slow=False)
    if filename is None:
      filename = f'{temp.name}.mp3'
    print(os.path.abspath(filename))
    tts.save(filename)
    mixer.init()
    mixer.music.load(filename)
    mixer.music.play()
    while mixer.music.get_busy():
      continue
    mixer.quit()


In [13]:

def transcribe(audio):
  global messages

  audio_file = open(audio, 'rb')
  print(audio_file)
  transcript = openai.Audio.transcribe('whisper-1',audio_file)
  print(transcript['text'])

  messages.append({"role":"user","content":transcript['text']})
  response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
  )

  system_message = response['choices'][0]['message']['content']
  # say(system_message,"system.mp3")
  subprocess.call(['say', system_message, '-o', 'system.aiff'])
  subprocess.call(['ffmpeg','-y','-i', 'system.aiff', '-acodec', 'pcm_s16le', '-ac', '1', '-ar', '16000', 'system.wav'])

  messages.append({"role":"assistant","content":system_message})

  chat_transcript = ""
  for msg in messages:
    if msg['role'] != 'system':
      chat_transcript += f'{msg["role"]}: {msg["content"]}\n\n'

  return chat_transcript

ui = gr.Interface(
  fn=transcribe,
  inputs=gr.Audio(source="microphone", type="filepath"),
  outputs=[gr.Audio('system.wav'),"text"],
)

ui.launch()

Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


In [15]:

ui = gr.Interface(
  fn=transcribe,
  inputs=gr.Audio(source="microphone", type="filepath",streaming=True),
  # outputs=[gr.Audio('system1.wav'),"text"],
  outputs=["html"]
)

ui.launch()

Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


<_io.BufferedReader name='/private/var/folders/vk/rlpkh5414m95phky_z9_d7pm0000gn/T/audio54da8659c898165b37ccd49761b4e692889443e6-0-100.wav'>
Thank you.


ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/5.1.2_5 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack